In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.pipeline import Pipeline
from itertools import compress
from sklearn.preprocessing import StandardScaler
%matplotlib notebook
stemmer = PorterStemmer()
tokenizer = RegexpTokenizer(r'\w+')
train_path = '/content/drive/MyDrive/TechSoc Submission/train.csv'
test_path = '/content/drive/MyDrive/TechSoc Submission/test.csv'
stopwords_path = '/content/drive/MyDrive/TechSoc Submission/stopwords.txt'
save_path='/content/drive/MyDrive/TechSoc Submission/final.csv'
model_path='/content/drive/MyDrive/TechSoc Submission/MyModel.pkl'
stopwords = []
file = open(stopwords_path, "r")
for line in file:
      word=line.strip()
      stopwords.append(stemmer.stem(word))
def PreProcesser(df):
    df = df.drop(columns = ['title','uid'])
    df['TolWords']= df['content'].apply(lambda x: len(x.split(' ')))
    df['TolSentance']=df['content'].apply(lambda x: len(x))
    df['AvgWordLen']= df['content'].apply(lambda x: np.mean([len(word)  for word in x.split(' ') ]))
    df['TotStopwords']= df['content'].apply(lambda x:len(list(compress(x.split(),[word in stopwords  for word in x.split()]))))
    df['content']=df['content'].apply(lambda x: TreebankWordDetokenizer().detokenize( stemmer.stem(word) for word in tokenizer.tokenize(x) ))
    df['content']= df['content'].apply(lambda x:TreebankWordDetokenizer().detokenize(list(compress(x.split(), [word not in stopwords  for word in x.split()]))))
    df['TolWordsFin']= df['content'].apply(lambda x: len(x.split(' ')))
    df['TolSentanceFin']=df['content'].apply(lambda x: len(x))
    df['AvgWordLenFin']= df['content'].apply(lambda x: np.mean([len(word)  for word in x.split(' ') ]))
    return df


# import required libraries

In [7]:
from dill import load_session
load_session(model_path)
df_pred = pd.read_csv(test_path)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.24.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MLPClassifier from version 0.24.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.24.2 when using version 0.22.2.post1. This might lead to breaking code or invalid result

Mounted at /content/drive


In [8]:
%%time
print("Code for Inference")
df_pred = PreProcesser(df_pred)
t = Vectorizer.transform(df_pred['content'])
t1=t[:,ind]
df_f=pd.DataFrame.sparse.from_spmatrix(t1)
df_fin = pd.concat([df_pred,df_f],axis=1)
X_pred = df_fin.drop(columns=['content'])
X_pred = scaler.transform(X_pred)
pred = clf.predict(X_pred)
# Full Code for running your inference (including any preprocessing you need to do on the test set)
# In this Cell as a comment also mention the CPU and GPU of the system you are using to run this inference
# CPU:  Standard google colab notebook:   
# GPU:  Standard google colab notebook:  

Code for Inference
CPU times: user 56.9 s, sys: 418 ms, total: 57.3 s
Wall time: 56.8 s


In [10]:
df_fin = pd.DataFrame(pred,columns=['target_ind'])
df_fin.index.name='uid'
df_fin.to_csv(save_path,index=True)